In [1]:
import xarray as xr
import hvplot.xarray
import hvplot.pandas
import numpy as np
import pandas as pd
import holoviews as hv
import geoviews as gv
import matplotlib as mpl
import geopandas as gpd
from pathlib import Path

import warnings

warnings.filterwarnings('ignore')

hv.extension('bokeh')

## Select the reservoir

In [20]:
SAVE = True
RESERVOIR = '0810'

start_date = pd.to_datetime('2023-07-21')  # clip to data only after science orbit began.

In [21]:
val_pts = gpd.read_file(Path('../data/validation-locations/2023-24-insitu-pts.geojson'))
val_polys = gpd.read_file(Path('../data/validation-locations/2023-24-insitu-poly.geojson'))

selected_reservoirs = val_pts['tmsos_id'].tolist()
res_names = val_pts[['tmsos_id', 'name']].set_index('tmsos_id').to_dict()['name']

RESERVOIR_NAME = res_names[RESERVOIR]
print(f'{RESERVOIR}: {RESERVOIR_NAME}')

val_res_pt = val_pts.loc[val_pts['tmsos_id'].isin(selected_reservoirs)]
val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

val_res_pt = val_pts.loc[val_pts['tmsos_id'].isin(selected_reservoirs)]
val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

capacity = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['CAP_MCM'].values[0])

global_map = (
    val_res_pt.hvplot(
        geo=True, tiles='OSM'
    ) * val_res_pt[val_res_pt['tmsos_id'] == RESERVOIR].hvplot(
        geo=True, color='red', size=100, 
    )
).opts(
    title=f"Locations of validation reservoirs. {RESERVOIR_NAME}, highlighted in red"
)

global_map

0810: Noi, Th


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]
   .Points.II :Points   [Longitude,Latitude]

In [22]:
(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR].hvplot(
    geo=True, tiles='OSM', shared_axes=False
)).opts(title=f"{RESERVOIR_NAME}")

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [23]:
BUFFER_M = 800 # m

utm_crs = val_polys[val_polys['tmsos_id'] == RESERVOIR].estimate_utm_crs()
print(f"UTM area of use: \n{utm_crs.area_of_use}")

buffered_roi_utm = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR].to_crs(utm_crs).buffer(BUFFER_M).to_crs('epsg:4326')
buffered_roi = buffered_roi_utm.to_crs('epsg:4326')

UTM area of use: 
- name: Between 102°E and 108°E, northern hemisphere between equator and 84°N, onshore and offshore. Cambodia. China. Indonesia. Laos. Malaysia - West Malaysia. Mongolia. Russian Federation. Singapore. Thailand. Vietnam.
- bounds: (102.0, 0.0, 108.0, 84.0)


In [24]:
gd_track_fn = Path('../data/swot_orbit/swot_orbit.geojson')
gd_track = gpd.read_file(gd_track_fn)
gd_track

,ID_PASS,START_TIME,geometry
0,1,Day 01 00:00:00,"MULTIPOLYGON (((-75.4587 -77.45455, -75.73434 ..."
1,2,Day 01 00:51:30,"MULTIPOLYGON (((84.56768 77.75119, 84.6008 78...."
2,2,Day 01 00:51:30,"MULTIPOLYGON (((-180 -52.37417, -180 -50.92787..."
3,3,Day 01 01:42:50,"MULTIPOLYGON (((-110.19573 -77.57156, -110.161..."
4,4,Day 01 02:34:20,"MULTIPOLYGON (((57.70091 77.75254, 57.69643 78..."
...,...,...,...
845,582,Day 21 18:10:40,"MULTIPOLYGON (((136.37126 77.75111, 136.40526 ..."
846,582,Day 21 18:10:40,"MULTIPOLYGON (((-180 72.80747, -180 73.45033, ..."
847,583,Day 21 19:02:00,"MULTIPOLYGON (((-58.39258 -77.57164, -58.3595 ..."
848,584,Day 21 19:53:30,"MULTIPOLYGON (((109.50449 77.75255, 109.5009 7..."


In [25]:
gd_nadir_fn = Path('../data/swot_orbit/nadir/swot_science_orbit_sept2015-v2_nadir.shp')
gd_nadir = gpd.read_file(gd_nadir_fn)
gd_nadir.head()

,ID_PASS,START_TIME,geometry
0,1,Day 01 00:00:00,"LINESTRING (-75.40128 -77.54325, -67.74948 -77..."
1,2,Day 01 00:51:30,"LINESTRING (84.56135 77.66162, 92.5227 77.5155..."
2,2,Day 01 00:51:30,"LINESTRING (-180 -52.65544, -179.4133 -53.7198..."
3,3,Day 01 01:42:50,"LINESTRING (-110.20279 -77.66112, -107.53758 -..."
4,4,Day 01 02:34:20,"LINESTRING (57.70176 77.66297, 60.37721 77.650..."


In [26]:
buffered_roi_gdf = gpd.GeoDataFrame(buffered_roi).rename({0: 'geometry'}, axis=1) # .hvplot(alpha=0.2, lw=2)
buffered_roi_gdf.set_geometry('geometry', inplace=True)

In [27]:
import hvplot.pandas
import geoviews as gv
from holoviews import opts
import matplotlib as mpl
from shapely.geometry import box


gd_track_subset = gd_track[gd_track.intersects(buffered_roi.geometry.values[0])]
gd_nadir_subset = gd_nadir[gd_nadir.ID_PASS.isin(gd_track_subset.ID_PASS)]
gd_nadir_20km = gd_nadir_subset.to_crs(utm_crs).buffer(10000, cap_style='flat').to_crs('epsg:4326')
gd_nadir_20km = gpd.GeoDataFrame(gd_nadir_20km)
gd_nadir_20km = gd_nadir_20km.rename({0: 'geometry'}, axis=1)
gd_nadir_20km.set_geometry('geometry', inplace=True)

roi_gdf = gpd.GeoDataFrame(buffered_roi).rename({0: 'geometry'}, axis=1)
roi_gdf = roi_gdf.set_geometry('geometry')
roi_gdf['name'] = RESERVOIR_NAME
roi_hv = roi_gdf.hvplot(geo=True, color='gray')

plot_lims = buffered_roi.buffer(0.5).bounds
minx = plot_lims.values[0][0]
maxx = plot_lims.values[0][2]
miny = plot_lims.values[0][1]
maxy = plot_lims.values[0][3]

xlim=(minx, maxx)
ylim=(miny, maxy)
view_bounds = box(xlim[0], ylim[0], xlim[1], ylim[1])

cmap = mpl.colormaps['Set1'].resampled(len(gd_track_subset['ID_PASS'].unique()))
gd_track_subset['color'] = [mpl.colors.rgb2hex(cmap(i)) for i in np.linspace(0, 1, len(gd_track_subset.groupby('ID_PASS')))]
tracks_hv = gd_track_subset.clip(view_bounds).hvplot(
    geo=True, tiles='OSM', 
    color=gd_track_subset['color'], 
    alpha=0.5, line_width=2,
    xlim=xlim,
    ylim=ylim,
    tools=['hover'], hover_cols=['ID_PASS']
)

nadir_hv = gd_nadir_20km.hvplot(
    geo=True, alpha=0.5, line_width=2, color='white', 
    # xlim=(int(buffered_roi.total_bounds[0])-1, int(buffered_roi.total_bounds[2])+1),
    # ylim=(int(buffered_roi.total_bounds[1])-1, int(buffered_roi.total_bounds[3])+1),
    xlim = (minx, maxx),
    ylim = (miny, maxy)
)

(tracks_hv * nadir_hv * roi_hv).opts(
    title=f"SWOT Ground Tracks over {RESERVOIR_NAME}", width=400
)

:Overlay
   .WMTS.I       :WMTS   [Longitude,Latitude]
   .Polygons.I   :Polygons   [Longitude,Latitude]   (_color,ID_PASS)
   .Polygons.II  :Polygons   [Longitude,Latitude]
   .Polygons.III :Polygons   [Longitude,Latitude]

In [28]:
gd_track_subset

,ID_PASS,START_TIME,geometry,color
128,90,Day 04 04:18:50,"MULTIPOLYGON (((25.579 77.75051, 25.61941 78.1...",#e41a1c
432,299,Day 11 15:31:10,"MULTIPOLYGON (((19.12978 -77.57326, 19.12453 -...",#ff7f00
838,577,Day 21 13:53:20,"MULTIPOLYGON (((19.50651 -77.57235, 19.53153 -...",#999999


### load data

In [29]:
# load
from pathlib import Path
import pandas as pd

swot_save_dir = Path('../data/swot')
swot_save_dir.mkdir(exist_ok=True)

dsses = []
reservoirs = []
available = []
platforms = []
times = []
ds_dict = {reservoir_id: None for reservoir_id in selected_reservoirs}

for reservoir_id in selected_reservoirs:
    fp = swot_save_dir / f'{reservoir_id}.nc'

    if not fp.exists():
        reservoirs.append(reservoir_id)
        dsses.append(None)
        available.append(False)
        platforms.append(None)
        times.append(None)
        continue
    
    ds = xr.open_dataset(fp, chunks='auto', engine='netcdf4', decode_coords='all', )
    dsses.append(ds)
    reservoirs.extend([reservoir_id] * len(ds.time))
    available.extend([True] * len(ds.time))
    platforms.extend(['swot'] * len(ds.time))
    times.extend(ds.time.values)

df = pd.DataFrame({
    'reservoir': reservoirs,
    'available': available,
    'platform': platforms,
    'time': times,
})

ds_dict = {reservoir_id: ds for reservoir_id, ds in zip(selected_reservoirs, dsses)}
df = pd.merge(df, val_res_poly[['tmsos_id', 'geometry']], left_on='reservoir', right_on='tmsos_id', suffixes=(False, False))
df.head()

,reservoir,available,platform,time,tmsos_id,geometry
0,0871,True,swot,2023-11-23,0871,"MULTIPOLYGON (((-122.68924 41.03545, -122.6889..."
1,0871,True,swot,2023-12-04,0871,"MULTIPOLYGON (((-122.68924 41.03545, -122.6889..."
2,0871,True,swot,2024-01-03,0871,"MULTIPOLYGON (((-122.68924 41.03545, -122.6889..."
3,0871,True,swot,2024-01-15,0871,"MULTIPOLYGON (((-122.68924 41.03545, -122.6889..."
4,0871,True,swot,2024-01-24,0871,"MULTIPOLYGON (((-122.68924 41.03545, -122.6889..."


## plot data for a single reservoir

In [30]:
ds = ds_dict[RESERVOIR]
ds

<xarray.Dataset> Size: 162MB
Dimensions:            (time: 56, y: 492, x: 210)
Coordinates:
  * x                  (x) float64 2kB 5.31e+05 5.311e+05 ... 5.519e+05
  * y                  (y) float64 4kB 1.634e+06 1.634e+06 ... 1.683e+06
    reservoir          <U4 16B ...
    spatial_ref        int64 8B ...
    crs                int64 8B ...
  * time               (time) datetime64[ns] 448B 2023-07-31 ... 2024-10-25
Data variables:
    water_area         (time, y, x) float32 23MB dask.array<chunksize=(56, 492, 210), meta=np.ndarray>
    water_area_qual    (time, y, x) float32 23MB dask.array<chunksize=(56, 492, 210), meta=np.ndarray>
    water_frac         (time, y, x) float32 23MB dask.array<chunksize=(56, 492, 210), meta=np.ndarray>
    water_frac_uncert  (time, y, x) float32 23MB dask.array<chunksize=(56, 492, 210), meta=np.ndarray>
    wse                (time, y, x) float32 23MB dask.array<chunksize=(56, 492, 210), meta=np.ndarray>
    wse_uncert         (time, y, x) float32 23MB dask.array<chunksize=(56, 492, 210), meta=np.ndarray>
    wse_qual           (time, y, x) float32 23MB dask.array<chunksize=(56, 492, 210), meta=np.ndarray>
    pass_ids           (time) <U3 672B dask.array<chunksize=(56,), meta=np.ndarray>
Attributes: (12/49)
    Conventions:                   CF-1.7
    title:                         Level 2 KaRIn High Rate Raster Data Product
    institution:                   JPL
    source:                        Ka-band radar interferometer
    history:                       2024-06-21T12:44:52Z : Creation
    platform:                      SWOT
    ...                            ...
    mgrs_latitude_band:            P
    x_min:                         505100.0
    x_max:                         651800.0
    y_min:                         1635000.0
    y_max:                         1781600.0
    product_version:               01

## Classify water for single reservoir

In [31]:
import rioxarray

ds = ds.rio.set_spatial_dims('y', 'x')
ds = ds.rio.write_crs(utm_crs)
ds = ds.rio.clip(buffered_roi.to_crs(ds.rio.crs).geometry.values)
ds = ds.sel(time=slice(start_date, None))
ds

<xarray.Dataset> Size: 136MB
Dimensions:            (x: 186, y: 468, time: 56)
Coordinates:
  * x                  (x) float64 1kB 5.322e+05 5.323e+05 ... 5.507e+05
  * y                  (y) float64 4kB 1.635e+06 1.636e+06 ... 1.682e+06
    reservoir          <U4 16B ...
    crs                int64 8B ...
  * time               (time) datetime64[ns] 448B 2023-07-31 ... 2024-10-25
    spatial_ref        int64 8B 0
Data variables:
    water_area         (time, y, x) float32 19MB dask.array<chunksize=(56, 468, 186), meta=np.ndarray>
    water_area_qual    (time, y, x) float32 19MB dask.array<chunksize=(56, 468, 186), meta=np.ndarray>
    water_frac         (time, y, x) float32 19MB dask.array<chunksize=(56, 468, 186), meta=np.ndarray>
    water_frac_uncert  (time, y, x) float32 19MB dask.array<chunksize=(56, 468, 186), meta=np.ndarray>
    wse                (time, y, x) float32 19MB dask.array<chunksize=(56, 468, 186), meta=np.ndarray>
    wse_uncert         (time, y, x) float32 19MB dask.array<chunksize=(56, 468, 186), meta=np.ndarray>
    wse_qual           (time, y, x) float32 19MB dask.array<chunksize=(56, 468, 186), meta=np.ndarray>
    pass_ids           (time) <U3 672B dask.array<chunksize=(56,), meta=np.ndarray>
Attributes: (12/49)
    Conventions:                   CF-1.7
    title:                         Level 2 KaRIn High Rate Raster Data Product
    institution:                   JPL
    source:                        Ka-band radar interferometer
    history:                       2024-06-21T12:44:52Z : Creation
    platform:                      SWOT
    ...                            ...
    mgrs_latitude_band:            P
    x_min:                         505100.0
    x_max:                         651800.0
    y_min:                         1635000.0
    y_max:                         1781600.0
    product_version:               01

In [34]:
from bokeh.models import FixedTicker
import geoviews as gv

# Create the satellite imagery panel
satellite_imagery = gv.tile_sources.OSM().opts(
    width=450, height=450, title='Satellite Imagery'
) * buffered_roi_gdf.hvplot(geo=True, color='red', alpha=0.1).opts(width=450, height=450)

# Create the water area plot
water_area_hv = ds['water_area'].sel(time='2024-01-30', method='nearest').hvplot(
    geo=True, clim=(0, 15000)
).opts(
    width=300, title='Water Surface Area of Dumboor Reservoir \nobserved by SWOT on 10th February, 2024',
    xlabel='Longitude', ylabel='Latitude',
    colorbar_opts={'title': 'Water Area (m²)'}, show_grid=True
)

# Create the water area quality plot
water_area_qual_hv = ds['water_area_qual'].sel(time='2024-01-30', method='nearest').hvplot(
    geo=True, cmap=['green', 'yellow', 'orange', 'red'], color_levels=[0, 1, 2, 3, 4], clim=(-0.5,3.5)
).opts(
    width=300, title='Water Classification Quality flag',
    colorbar_opts={
        'title': 'Quality', 
        'ticker': FixedTicker(ticks=[0, 1, 2, 3]), 
        'major_label_overrides': {0: 'good', 1: 'suspect', 2: 'degraded', 3: 'bad measurement'}
    },
    xlabel='Longitude', ylabel='Latitude', show_grid=True
)

# Create the water fraction uncertainty plot
water_uncert_hv = ds['water_frac_uncert'].sel(time='2024-01-30', method='nearest').hvplot(
    geo=True, 
).opts(
    width=300, clim=(0, 1), title='Water Fraction Uncertainty (0-1)',
    xlabel='Longitude', ylabel='Latitude', cmap='reds', show_grid=True
)

# Combine the plots into a 2-column layout
combined_plot = (satellite_imagery + water_area_hv + water_area_qual_hv + water_uncert_hv).cols(2)
combined_plot

:Layout
   .Overlay.I :Overlay
      .WMTS.I     :WMTS   [Longitude,Latitude]
      .Polygons.I :Polygons   [Longitude,Latitude]
   .Image.I   :Image   [x,y]   (water surface area)
   .Image.II  :Image   [x,y]   (summary quality indicator for the water surface area)
   .Image.III :Image   [x,y]   (uncertainty in the water fraction)

In [35]:
xy_mask = ds['wse_uncert'] < 0.1
uncertainty_masked_wse = ds['wse'].where(xy_mask).chunk(dict(x=-1, y=-1))

# Calculate the 25% and 75% IQR elevations
elevations = uncertainty_masked_wse.median(dim=('x', 'y')).values

q25 = uncertainty_masked_wse.quantile(0.25, dim=('x', 'y')).values
q75 = uncertainty_masked_wse.quantile(0.75, dim=('x', 'y')).values  # Corrected quantile to 0.75

time_mask = (q75-q25) > 0.5  # mask out values where IQR distance is > 0.5 m (50 cm)

elevations = np.where(time_mask, np.nan, elevations)

pass_ids = ds['pass_ids'].values
time = ds['time'].values

# Calculate the fraction of values left after masking
values_left_pct = sum(~np.isnan(elevations))*100/len(time)
fraction = f"{sum(~np.isnan(elevations))}/{len(time)}"

# Create the scatter plot
scatter = hv.Scatter(
    (time, elevations, pass_ids), 
    kdims=["time", "elevation"], 
    vdims="pass_ids"
).opts(
    color="pass_ids", colorbar=True, size=8, width=600, show_grid=True,
    cmap='Set1', title=f"{RESERVOIR}: {RESERVOIR_NAME}\nCapacity: {capacity} mil. m3.\n{values_left_pct:.1f}% values left after masking\n{fraction} values left after masking"
)

error_bars = hv.ErrorBars(
    (time, elevations, elevations-q25, q75-elevations)
).opts(
    color='black', line_width=1.5, width=600
)

fig = error_bars * scatter
fig

:Overlay
   .ErrorBars.I :ErrorBars   [x]   (y,yerror)
   .Scatter.I   :Scatter   [time,elevation]   (pass_ids)

In [36]:
# save elevations
elevation_save_dir = Path("../data/elevation/swot_karin")
save_fp = elevation_save_dir / f"{RESERVOIR}.csv"

elevations
time
# Create a DataFrame with elevations and time
elevation_df = pd.DataFrame({
    'time': time,
    'elevation': elevations
})

# Save the DataFrame to a CSV file
elevation_df.to_csv(save_fp, index=False)

##### end of notebook